# Parsing Social Media API's into CSV's

### General: 

In [122]:
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np

import re

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import matplotlib.pyplot as plt

In [123]:
def remove_special_characters(text):
    """
    Remove all special characters from the given text.
    :param text: The input text string.
    :return: Cleaned text string with only alphanumeric characters and spaces.
    """
    if isinstance(text, str):  # Check if the input is a string
        text = text.replace("&#39;", "'").replace("�", "")
        cleaned_text = re.sub(r'[^A-Za-z0-9\s]+', '', text)
        return cleaned_text
    return '' 


In [124]:

# List of cyberbullying keywords (can be expanded)
cyberbullying_keywords = [
    "loser", "idiot", "stupid", "ugly", "kill", "dumb", "hate", "trash", 
    "fat", "moron", "freak", "retard", "bitch"
]

# Text speak dictionary to expand common abbreviations
text_speak_dict = {
    "u": "you",
    "ur": "your",
    "lol": "laugh out loud",
    "omg": "oh my god",
    "lmao": "laughing my ass off",
    "rofl": "rolling on the floor laughing",
    "btw": "by the way",
    "tbh": "to be honest",
    "idk": "i don't know",
    "smh": "shaking my head",
    "gr8": "great",
    "wtf": "what the f***",
    "imo": "in my opinion",
    "fml": "f*** my life"
    # Add more as needed
}


In [125]:

# Function to expand text speak using regex
def expand_text_speak(text):
    """
    Replace text speak abbreviations with their full forms using regex.
    :param text: The input text string.
    :return: Text with expanded abbreviations.
    """
    if isinstance(text, str):  # Check if the input is a string
        # Replace each text speak abbreviation using the dictionary
        for abbrev, full_form in text_speak_dict.items():
            text = re.sub(rf'\b{abbrev}\b', full_form, text, flags=re.IGNORECASE)
        return text
    return ''


In [126]:

# Function to label comments as "Cyberbullying" or "Not Cyberbullying"
def sentiment_scores(comment, polarity):

	# Creating a SentimentIntensityAnalyzer object.
	sentiment_object = SentimentIntensityAnalyzer()

	sentiment_dict = sentiment_object.polarity_scores(comment)
	polarity.append(sentiment_dict['compound'])

	return polarity

# Function to analyze the sentiment of a single comment and return its label
def sentiment_analyzer(comment):
    sentiment_object = SentimentIntensityAnalyzer()
    sentiment_dict = sentiment_object.polarity_scores(comment)

    # Use compound score to determine if the comment is cyberbullying
    polarity = sentiment_dict['compound']

    if polarity > 0.05:
        return "Not Cyberbullying"
    elif polarity < -0.05:
        return "Cyberbullying"
    else:
        return "Neutral"

	
def label_comment(text):
    # Clean the text by removing special characters and expanding text speak
    cleaned_text = remove_special_characters(text)
    expanded_text = expand_text_speak(cleaned_text)

    # Check for any cyberbullying keywords in the expanded text
    return sentiment_analyzer(expanded_text)



In [127]:

# Function to read comments from CSV, label them, and save to a new CSV
def label_comments_from_csv(input_csv: str, output_csv: str):
    """
    Read comments from a CSV, label them, and save the labeled data to a new CSV.
    
    :param input_csv: Path to the input CSV file containing comments.
    :param output_csv: Path to the output CSV file for labeled comments.
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(input_csv)

    if df.empty:
        raise ValueError("csv is empty")
    
    df.columns = ['Comment']
    # Apply labeling function to each comment
    df['Label'] = df['Comment'].apply(label_comment)
    df['Comment'] = df['Comment'].apply(remove_special_characters)
    # Save the labeled data to a new CSV file
    df.to_csv(output_csv, index=False)
    print(f"Labeled comments saved to {output_csv}")


In [128]:
df_list = []
csv_files = ["data/AhmedScraped.csv", "data/JacksonScraped.csv","data/CyberbullyingCmts_5000.csv", "data/SoniaScraped.csv"]
for file_path in csv_files:
    df = pd.read_csv(file_path)  # Read the CSV file
    df_list.append(df)  # Append the DataFrame to the list

# Concatenate all DataFrames in the list into one DataFrame
merged_df = pd.concat(df_list, ignore_index=True)
df_t = merged_df.apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
df_t.to_csv('data/merged_output.csv', index=False)
df_t.describe()

count     25934
unique    21905
top           💩
freq         44
dtype: object

In [129]:
input_csv_file = 'data/merged_output.csv'  # Path to your existing comments CSV
output_csv_file = 'data/labeled_comments.csv'  # Path to save the labeled comments
label_comments_from_csv(input_csv_file, output_csv_file)
df = pd.read_csv(output_csv_file)
df.head()

Labeled comments saved to data/labeled_comments.csv


,Comment,Label
0,they never told me I couldnt sing what in the ...,Neutral
1,THEY DIDNT SAY I COULDNT SIIINGGGGG kat from v...,Neutral
2,Youre very brave for having the comment sectio...,Not Cyberbullying
3,i didnt know what her response would be but i ...,Neutral
4,This is so insulting to the victims,Cyberbullying


In [130]:
df.describe()


,Comment,Label
count,25621,25934
unique,21591,3
top,Two steps ahead,Not Cyberbullying
freq,32,9077


In [131]:
df['Comment'].fillna("", inplace=True)
df['Comment'] = df['Comment'].astype(str)
def keyword_presence(text):
    text = text.lower()  # Convert to lowercase for case-insensitive matching
    return {keyword: int(keyword in text) for keyword in cyberbullying_keywords}

# Apply the function to create new binary columns
keyword_columns = df['Comment'].apply(keyword_presence).apply(pd.Series)

# Concatenate the new binary columns to the original DataFrame
df = pd.concat([df, keyword_columns], axis=1)

C:\Users\ERC\AppData\Local\Temp\ipykernel_34060\3703677236.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Comment'].fillna("", inplace=True)


In [132]:
columns = [col for col in df.columns if col != 'Label'] + ['Label']
df = df[columns]

In [133]:
df.head()
df.to_csv('data/word_to_column.csv', index=False)

In [134]:
import pandas as pd

def count_labels(csv_file: str):
    # Load the labeled CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Ensure the 'Label' column exists
    if 'Label' not in df.columns:
        raise ValueError("The CSV does not contain a 'Label' column.")

    # Count occurrences of each label ('Cyberbullying', 'Not Cyberbullying', 'Neutral')
    label_counts = df['Label'].value_counts()

    # Print the counts
    print(f"Label Counts:\n{label_counts}")

    return label_counts

# Example usage:
csv_file = "data/labeled_comments.csv"  # Path to your labeled CSV file
count_labels(csv_file)


Label Counts:
Label
Not Cyberbullying    9077
Neutral              8707
Cyberbullying        8150
Name: count, dtype: int64


Label
Not Cyberbullying    9077
Neutral              8707
Cyberbullying        8150
Name: count, dtype: int64